# AI Exam (Ex 3)

An advanced aquatic drone is deployed to collect critical data on marine biodiversity in a coastal region. The drone starts at point $S$, located near the shore, and must navigate to point $G$, a designated marine research site rich in coral reefs and sea life. Along the way, the drone must carefully maneuver through dynamic underwater environments, avoiding hazards and optimizing energy usage.

The environment includes:  
1. **(O) Open Water:** Normal movement; no additional challenges.  
2. **(C) Currents:** Areas where ocean currents influence the drone's movement, potentially pushing it off course.  
3. **(F) Seaweed Forests:** Dense vegetation that slows the drone, incurring extra energy costs per move.    
4. **(R) Rocky Areas:** Dangerous zones that the drone must avoid, we do not know what happens to the agent if it enters in the cell.  
5. **(E) Energy Stations:** Specific points where the drone can recharge its battery, adding a small reward to encourage efficient navigation.  


<img src="images/env_ex3.png" style="zoom: 20%;"/>



### Environment Details:
The partial information on the enviroment you have are:
- **Grid Representation:** The above image represents the environment (a grid 10x10).  
- $S$ - Start state: The drone's starting point at (0, 0).  
- $G$ - Goal state: The marine research site at (9, 7) providing a large positive reward +20.0 and ending the episode.  
- **Movement Costs:** Each move has a default energy cost of -0.01.  
- **Hazards:**  
  - **Strong Currents:** Entering a current zone imposes a stochastic movement:  
    - 80% chance to move as intended.  
    - 10% chance to be pushed one cell in the left direction of the current.  
    - 10% chance to be pushed one cell in the right direction of the current. 
  - **Seaweed Forests:** Entering these zones incurs an additional -0.1 reward penalty.
  - **Rocky areas:** For these areas the model for the dynamics of the drones and the reward is not available
- **Energy Stations:** Provide a 0.1 reward when visited, (however reaching these cells may require the agent to move far from the goal).   

Consider the problem of computing the optimal solution for the environment reported above. You can interact with the environment using the code reported here below: 

In [ ]:
import os, sys 
import tqdm

module_path = os.path.abspath(os.path.join('tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym, envs
from utils.ai_lab_functions import *
import numpy as np
from timeit import default_timer as timer
from tqdm import tqdm as tqdm

env_name = 'AquaticEnv-v0'
env = gym.make(env_name)

env.render()

# you can visualize the policy of a solution by calling the plot_policy function
policy = np.random.choice(list(env.actions.keys()), env.observation_space.n)
visual_policy = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols)) 
plot_policy(visual_policy)

In [ ]:
print("\nActions encoding: ", env.actions)

# Remember that you can know the type of a cell whenever you need by accessing the grid element of the environment:
print("Cell type of start state: ",env.grid[env.startstate])
print("Cell type of goal state: {} Reward: {}".format(env.grid[env.goalstate],env.RS[env.goalstate]))

state = 15 # normal state
print(f"Cell type of cell {env.state_to_pos(state)}: ",env.grid[state])
print(f"Probability of effectivelty performing action {env.actions[0]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[0], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[1]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[1], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[2]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[2], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[3]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[3], state+1]}")

state = 10 # state with stochastic transitions
print(f"\nCell type of cell {env.state_to_pos(state)}: ",env.grid[state])
print(f"Probability of effectivelty performing action {env.actions[0]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[0], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[1]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[1], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[2]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[2], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[3]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[3], state+1]}")

state = 10 # state with stochastic transitions
print(f"\nCell type of cell {env.state_to_pos(state)}: ",env.grid[state])
print(f"Probability of effectivelty performing action {env.actions[0]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[0], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[1]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[1], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[2]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[2], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[3]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[3], state+1]}")


#### Q1: Provide an algorithm that finds a solution that reaches the goal without touching any 'R' state and passing through at least one charging station in this environment. Use the approach that you think is most appropriate motivating your choice.

In [ ]:
env_name = 'AquaticEnv-v0'
env = gym.make(env_name)

# for reproducibility purposes you can set a seed to your algorithm. For instance:
seed = 0
#sol, _, _ = your_method(env, your_hyperparams, seed=0)
# inside the function then
def your_method(env, hyperparams, seed=None):
    if seed is not None:
        np.random.seed(seed)
    # your code here
    return sol, _, _

# you can visualize the policy of a solution by calling the plot_policy function
#using a random policy for now, you should replace this with your solution 
sol = np.random.choice(list(env.actions.keys()), env.observation_space.n)
visual_policy = np.vectorize(env.actions.get)(sol.reshape(env.rows, env.cols)) 
plot_policy(visual_policy)

